In [4]:
# Dependencies
import os
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Pretty Print
from pprint import pprint

In [5]:
# Import our movie data
filepath = os.path.join("Resources","MovieData_Clean.csv")
df_IMDB = pd.read_csv(filepath)

# Display our dataframe
df_IMDB

,TMDB ID,Title,Release Date,Budget,Revenue,IMDB ID,Rating,Runtime,Genre,Director,...,Awards,Poster,IMDB Rating,IMDB Votes,DVD Release Date,Production,Day of Release,Month of Release,Year of Release,Revenue (Millions)
0,110681,Kings of the Evening,2008-01-01,0.0,98682.0,tt1006904,PG,99 min,Drama,Andrew P. Jones,...,8 wins & 1 nomination.,https://m.media-amazon.com/images/M/MV5BMjA3OT...,6.0,234,01 Nov 2010,Indican Pictures,Tuesday,January,2008,0.098682
1,26163,The Decline And Fall Of America,2008-01-01,7500.0,1200.0,tt1786696,NaN,120 min,Documentary,B.A. Brooks,...,NaN,https://m.media-amazon.com/images/M/MV5BMTMzNj...,5.2,13,NaN,NaN,Tuesday,January,2008,0.001200
2,13971,Wild Child,2008-01-01,20000000.0,21972336.0,tt1024255,PG-13,124 min,"Comedy, Drama, Romance",Nick Moore,...,NaN,https://m.media-amazon.com/images/M/MV5BNzUzYT...,6.1,"46,716",17 Nov 2009,Universal Pictures,Tuesday,January,2008,21.972336
3,6933,One Missed Call,2008-01-04,27.0,26.0,tt0479968,PG-13,87 min,"Horror, Mystery",Eric Valette,...,2 nominations.,https://m.media-amazon.com/images/M/MV5BMjA2NT...,4.0,"30,026",22 Apr 2008,Warner Bros. Pictures,Friday,January,2008,0.000026
4,14423,First Sunday,2008-01-11,0.0,38804615.0,tt0486578,PG-13,96 min,"Comedy, Crime, Drama",David E. Talbert,...,NaN,https://m.media-amazon.com/images/M/MV5BMjE4MT...,4.5,"9,203",13 May 2008,NaN,Friday,January,2008,38.804615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3314,512200,Jumanji: The Next Level,2019-12-13,125000000.0,615882314.0,tt7975244,PG-13,123 min,"Action, Adventure, Comedy, Fantasy",Jake Kasdan,...,2 nominations.,https://m.media-amazon.com/images/M/MV5BOTVjMm...,7.0,"61,847",NaN,Sony Pictures,Friday,December,2019,615.882314
3315,536869,Cats,2019-12-20,95000000.0,36980175.0,tt5697572,PG,110 min,"Comedy, Drama, Family, Fantasy, Musical",Tom Hooper,...,Nominated for 1 Golden Globe. Another 1 win & ...,https://m.media-amazon.com/images/M/MV5BNjRlNT...,2.8,"20,921",NaN,NaN,Friday,December,2019,36.980175
3316,522212,Just Mercy,2019-12-25,0.0,10435988.0,tt4916630,PG-13,136 min,Drama,Destin Daniel Cretton,...,7 wins & 8 nominations.,https://m.media-amazon.com/images/M/MV5BYmM4Yz...,6.4,"1,551",NaN,Warner Bros. Pictures,Wednesday,December,2019,10.435988
3317,448119,Dolittle,2020-01-17,175000000.0,61184730.0,tt6673612,PG,101 min,"Action, Adventure, Comedy, Family, Fantasy",Stephen Gaghan,...,NaN,https://m.media-amazon.com/images/M/MV5BMDNkOD...,5.6,"4,762",NaN,Universal Pictures,Friday,January,2020,61.184730


In [6]:
# Rename some values in our dataset
df_IMDB.replace({"Rating":{"NOT RATED":"NR", "Not Rated":"NR", "UNRATED":"NR", "Unrated":"NR"}}, inplace=True)

# Clean the data to remove nulls and zeros from the plotted columns
# First, replace the NaN cells with zeros
df_IMDB['Rating'] = df_IMDB['Rating'].fillna(0)
df_IMDB['Revenue'] = df_IMDB['Revenue'].fillna(0)

# Now drop rows with zeros in the plotted columns and strange values in the rating
df_IMDB_clean = df_IMDB[df_IMDB.Rating != 0]
df_IMDB_clean = df_IMDB_clean[df_IMDB_clean.Revenue != 0]
df_IMDB_clean = df_IMDB_clean[df_IMDB_clean.Rating != "12"]

# Display our dataframe
df_IMDB_clean

,TMDB ID,Title,Release Date,Budget,Revenue,IMDB ID,Rating,Runtime,Genre,Director,...,Awards,Poster,IMDB Rating,IMDB Votes,DVD Release Date,Production,Day of Release,Month of Release,Year of Release,Revenue (Millions)
0,110681,Kings of the Evening,2008-01-01,0.0,98682.0,tt1006904,PG,99 min,Drama,Andrew P. Jones,...,8 wins & 1 nomination.,https://m.media-amazon.com/images/M/MV5BMjA3OT...,6.0,234,01 Nov 2010,Indican Pictures,Tuesday,January,2008,0.098682
2,13971,Wild Child,2008-01-01,20000000.0,21972336.0,tt1024255,PG-13,124 min,"Comedy, Drama, Romance",Nick Moore,...,NaN,https://m.media-amazon.com/images/M/MV5BNzUzYT...,6.1,"46,716",17 Nov 2009,Universal Pictures,Tuesday,January,2008,21.972336
3,6933,One Missed Call,2008-01-04,27.0,26.0,tt0479968,PG-13,87 min,"Horror, Mystery",Eric Valette,...,2 nominations.,https://m.media-amazon.com/images/M/MV5BMjA2NT...,4.0,"30,026",22 Apr 2008,Warner Bros. Pictures,Friday,January,2008,0.000026
4,14423,First Sunday,2008-01-11,0.0,38804615.0,tt0486578,PG-13,96 min,"Comedy, Crime, Drama",David E. Talbert,...,NaN,https://m.media-amazon.com/images/M/MV5BMjE4MT...,4.5,"9,203",13 May 2008,NaN,Friday,January,2008,38.804615
5,7191,Cloverfield,2008-01-15,25000000.0,170764026.0,tt1060277,PG-13,85 min,"Horror, Sci-Fi, Thriller",Matt Reeves,...,5 wins & 30 nominations.,https://m.media-amazon.com/images/M/MV5BZDNhND...,7.0,"358,331",22 Apr 2008,Paramount,Tuesday,January,2008,170.764026
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3314,512200,Jumanji: The Next Level,2019-12-13,125000000.0,615882314.0,tt7975244,PG-13,123 min,"Action, Adventure, Comedy, Fantasy",Jake Kasdan,...,2 nominations.,https://m.media-amazon.com/images/M/MV5BOTVjMm...,7.0,"61,847",NaN,Sony Pictures,Friday,December,2019,615.882314
3315,536869,Cats,2019-12-20,95000000.0,36980175.0,tt5697572,PG,110 min,"Comedy, Drama, Family, Fantasy, Musical",Tom Hooper,...,Nominated for 1 Golden Globe. Another 1 win & ...,https://m.media-amazon.com/images/M/MV5BNjRlNT...,2.8,"20,921",NaN,NaN,Friday,December,2019,36.980175
3316,522212,Just Mercy,2019-12-25,0.0,10435988.0,tt4916630,PG-13,136 min,Drama,Destin Daniel Cretton,...,7 wins & 8 nominations.,https://m.media-amazon.com/images/M/MV5BYmM4Yz...,6.4,"1,551",NaN,Warner Bros. Pictures,Wednesday,December,2019,10.435988
3317,448119,Dolittle,2020-01-17,175000000.0,61184730.0,tt6673612,PG,101 min,"Action, Adventure, Comedy, Family, Fantasy",Stephen Gaghan,...,NaN,https://m.media-amazon.com/images/M/MV5BMDNkOD...,5.6,"4,762",NaN,Universal Pictures,Friday,January,2020,61.184730


In [7]:
# Get average revenue by rating
dfavgRev_ByRating = df_IMDB_clean.groupby(["Rating"])["Revenue"].mean().to_frame()
dfavgRev_ByRating.reset_index(inplace=True)
dfavgRev_ByRating.set_index("Revenue", inplace=True)
dfavgRev_ByRating

# Display the results
# ax = dfavgRev_ByRating[('Revenue', 'Rating')].plot(kind="bar", alpha=0.7)
# ax = dfavgRev_ByRating.plot.bar(title="Movie Rating v. Total Revenue (Millions)", legend=False)
# ax.set_ylim([0, 250])
# ax.set_xlabel("Total Revenue (Millions)")
# ax.set_ylabel("Movie Rating")
# ax.set_title("Movie Rating v. Total Revenue (Millions)")

# # # Show the chart
# plt.show()

# Save our figure
# save_file = os.path.join(output_dir, "Cities_LatvTemp.png")
# fig.savefig(save_file)

,Rating
Revenue,
4.343950e+06,Approved
2.742230e+08,G
1.359910e+06,GP
9.304933e+06,NC-17
1.491932e+07,NR
1.768345e+08,PG
1.614548e+08,PG-13
4.742382e+07,R
1.790000e+05,TV-13
